Metabolic Chamber Data Analysis: abx

In [ ]:
# Import pertinent packages

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

In [ ]:
# Set working directory
path = "/Users/ellacho/Desktop"
os.chdir(path)

In [ ]:
# Upload data.. only upload after deleting title rows
data = pd.read_csv('052523_aba_abx_EC_KK.csv', sep=',')
print(data)

Data Wrangling

In [ ]:
# Separate data by each box

boxdata = []
for i in range(0,9):
    boxdata.append(data.loc[data['Box'] == i])
    
# boxdata[#] gives you data per box; # = actual box number

Box data parameters:

SumTime: Total duration (mins) that the animal spent in the wheel <br>
SumR+L: Total number of wheel rotations (both left and right) <br>
SumRuns: Total number of run episodes (run = rpm > threshold set) <br>
MaxSpeed: Maximum speed (rpm) in each run episode <br>
AvgSpeed: Average speed of wheel (rpm) in all run episodes <br>
MaxLen: Longest duration of a run episode <br>
<br>
Note: Wheel diameter = 12.5cm

In [ ]:
# Create an array of dates

dates = np.array(["5/26/2023", "5/27/2023", "5/28/2023", "5/29/2023"])

In [ ]:
# Define a function to separate each box's data by date 

def boxdatafxn(boxdatabox, date_array):
    boxdata_bydate = []
    
    for i in range(len(date_array)):
        boxdata_bydate.append(boxdatabox.loc[boxdatabox['Date'] == date_array[i]])

    return boxdata_bydate

In [ ]:
# Iterate function over each box

# Create an empty list to store data
dates_boxdata = []

for i in range(len(boxdata)):
    dates_boxdata.append(boxdatafxn(boxdata[i], dates))
    
# Output: dates_boxdata[box number (1-8)][day (0-4)]['ColumnName']
# To get single rows of data: ie dates_boxdata[1][4]['Drink1'].iloc[row number]

Running wheel analysis: # of Runs

In [ ]:
# Create a function to find the sum of total wheel revolutions 

def sumrunfxn(dates_boxdata_array):
    
    sumruns_array = np.zeros(len(dates))
    
    for i in range(len(dates)):
        sumruns_array[i] = np.sum(dates_boxdata_array[i]['SumR+L'])
    
    return sumruns_array

In [ ]:
# Iterate sum function over each box

sumruns = []

for i in range(len(boxdata)):
    sumruns.append(sumrunfxn(dates_boxdata[i]))
    
# Notation: sumruns[box#] gives that box's summed runs per day 

Food intake and drink analysis

In [ ]:
# Create a function to measure food intake in each time interval (because measurements on raw data are cumulative)

def intake(date_box_array, intaketype):
    # Create an empty array to store time intervals and food consumed
    time_int = np.arange(0, 9 * len(date_box_array), 9) # units: Minutes since trial start
    food_int = np.zeros(len(date_box_array)) # units: Grams or mL consumed in each interval
    
    for i in range(len(date_box_array) - 1):
        food_int[i] = date_box_array[intaketype].iloc[i + 1] - date_box_array[intaketype].iloc[i]
        
    return time_int, food_int

# Output: ie foodintake(dates_boxdata[1][0], 'Drink1')[0] gives time array, [1] gives food consumed array

In [ ]:
# Create a function to obtain food intake and liquid intake for each box (for each day too)

def foodintakebox(dates_boxdata_array):
    waterintake_boxdata = []
    foodintake_boxdata = []
    
    for i in range(len(dates)):
        waterintake_boxdata.append(intake(dates_boxdata_array[i], 'Drink1'))
        foodintake_boxdata.append(intake(dates_boxdata_array[i], 'Feed1'))
    
    return waterintake_boxdata, foodintake_boxdata

# Output syntax: ie foodintakebox(dates_boxdata[box number])[0 or 1][day number 0-4]
    # [0] gives water, [1] gives food

In [ ]:
# Evaluate food intakes for each box 

intakedata = []

for i in range(len(boxdata)):
    intakedata.append(foodintakebox(dates_boxdata[i]))
    
# Output: foodintakedata[box number][breeder vs. chow 0-1][day number 0-4][time vs. intake 0-1]

In [ ]:
# Find the total amt of food consumed per day for each food and each mouse

def totalfood(foodintakebox_array):
    
    sum_breederintake_boxdata = []
    sum_chowintake_boxdata = []
    
    for i in range(len(dates)):
        sum_breederintake_boxdata.append(np.sum(foodintakebox_array[0][i][1]))
        sum_chowintake_boxdata.append(np.sum(foodintakebox_array[1][i][1]))
    
    return sum_breederintake_boxdata, sum_chowintake_boxdata

# Output syntax: ie totalfood(dates_boxdata[box number])[0 or 1][day number 0-4]
    # [0] gives breeder total, [1] gives chow total

In [ ]:
# Evaluate food intakes for each box 

foodintakedata = []

for i in range(len(boxdata)):
    foodintakedata.append(totalfood(intakedata[i]))
    
# Output:foodintakedata[box number][drink vs. chow 0-1][day number 0-4] gives the total intake of that day

In [ ]:
# Create a function to find the sum of food intake

def sumintakefxn(foodintakebox_array):
    
    sumfood_array = np.zeros(len(dates))
    sumdrink_array = np.zeros(len(dates))
    
    for i in range(len(dates)):
        sumfood_array[i] = np.sum(foodintakebox_array[1][i])
        sumdrink_array[i] = np.sum(foodintakebox_array[0][i])
        
    
    return sumfood_array, sumdrink_array

In [ ]:
# Iterate over all of the boxes 

sumintake = []

for i in range(len(boxdata)):
    sumintake.append(sumintakefxn(foodintakedata[i]))
    
# Notation: sumintake[box#][0 for food, 1 for drink]

Create dataframes for export

In [ ]:
# Create a dataframe of total runs per day

sumruns_df = pd.DataFrame({
"m1": sumruns[1],
"m2": sumruns[2],
"m3": sumruns[3],
"m4": sumruns[4],
"m5": sumruns[5],
"m6": sumruns[6],
"m7": sumruns[7],
"m8": sumruns[8]}, 
                       index = dates)

In [ ]:
# Create a dataframe of total food consumption per day

totalfood_df = pd.DataFrame({
"m1": sumintake[1][0],
"m2": sumintake[2][0],
"m3": sumintake[3][0],
"m4": sumintake[4][0],
"m5": sumintake[5][0],
"m6": sumintake[6][0],
"m7": sumintake[7][0],
"m8": sumintake[8][0]}, 
                       index = dates)

In [ ]:
# Create a dataframe of total drink consumption per day

totaldrink_df = pd.DataFrame({
"m1": sumintake[1][1],
"m2": sumintake[2][1],
"m3": sumintake[3][1],
"m4": sumintake[4][1],
"m5": sumintake[5][1],
"m6": sumintake[6][1],
"m7": sumintake[7][1],
"m8": sumintake[8][1]}, 
                       index = dates)

In [ ]:
# Export dataframes
sumruns_df.to_csv(r'/Users/ellacho/Downloads/sumruns_df_baseline.csv')
totalfood_df.to_csv(r'/Users/ellacho/Downloads/totalfood_df_baseline.csv')
totaldrink_df.to_csv(r'/Users/ellacho/Downloads/totaldrink_df_baseline.csv')